In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import time
import pickle

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import lightgbm as lgb

from datetime import datetime, timedelta
from sklearn.model_selection import KFold

from baseFunctions import *
from data_helpers import processData6, featureEngineering, getSequencesFast, removeOutliers, create_sequences, processAllData

#import dask.dataframe as dd
import plotly.express as px


In [ ]:
data, propDicts, flippedPropDicts = processData6()
data, timeFeatures = featureEngineering(data,splits=[2,2,2,2])

In [ ]:
initial_lag = 16 # 16 = independent from previous predictions
targetLags = [1,2,3,4,5,6,7]
featureLags = [1,2,3,4,5,6,7]
rolling = [7,21]

data, addedF = processAllData(data, targetLags, featureLags, rolling, initial_lag)

In [ ]:
addedF

In [ ]:
""" define features and test/train partition """
date_string_test = "2017-08-01"

train_bigDf = data.loc[(data.date < date_string_test) & (data.date > "2015-05-01") & (data.dataT =='train')]
test_bigDf  = data.loc[(data.date >= date_string_test) & (data.dataT !='test')]
val_bigDf   = data.loc[ data.dataT =='test']

trainF = [
       'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       'city', 'state', 'type', 'cluster', 
       'dcoilwtico', 
       'holidayType',
       'description', 
       'transferred', 
       #'transactions', 
       'store_closed']
timeF = [
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0','day_of_year_f24_0',  'day_of_year_f52_0',
       'day_of_year_f12_180', 'day_of_year_f104_180','day_of_year_f24_180','day_of_year_f52_180', 
       'weekday', 'month'
       ]

allF = addedF + trainF + timeF

In [ ]:
""" create lag 16 dataframe """
predictDiff16 = True
targetF, refF = 'target', 'ref'
logTransform16 = True
X_train16, X_test16, X_pred16 = train_bigDf[allF],test_bigDf[allF],val_bigDf[allF]
y_train16, y_test16 = train_bigDf[targetF], test_bigDf[targetF]
baseTrain16, baseTest16, basePred16 = train_bigDf[refF], test_bigDf[refF],val_bigDf[refF]

### create Sequences

In [ ]:
def getSequencePerGroup(group):
    X = group[allF]
    y = group['target']
    ref = group['ref']
    n_sequence = 16

    X0 = np.lib.stride_tricks.sliding_window_view(X, (n_sequence, len(allF)))[:,0,:,:]
    y0 = np.lib.stride_tricks.sliding_window_view(y, (n_sequence))#[:,0,:,:]
    ref0 = np.lib.stride_tricks.sliding_window_view(ref, (n_sequence))#[:,0,:,:]

    trainEnd = X0.shape[0] - 17
    testEnd = X0.shape[0]-1
    predEnd = X0.shape[0]

    X_trainM= X0[0:trainEnd,:,:]
    X_testM = X0[trainEnd:testEnd,:,:]
    X_valM =  X0[testEnd:predEnd,:,:]
    y_trainM= y0[0:trainEnd,:]
    y_testM = y0[trainEnd:testEnd,:]
    y_valM =  y0[testEnd:predEnd,:]
    base_trainM= ref0[0:trainEnd,:]
    base_testM = ref0[trainEnd:testEnd,:]
    base_valM =  ref0[testEnd:predEnd,:]

    return X_trainM, X_testM, X_valM, y_trainM, y_testM, y_valM, base_trainM, base_testM, base_valM

In [ ]:
ddf = dd.from_pandas(data, npartitions=4)  # Adjust the number of partitions as needed

# Step 2: Group the Dask DataFrame by the 'time_series_id' column
grouped = ddf.groupby('store_nbr','family')

# Step 3: Create a custom function to generate sequences
def create_sequences(group, seq_length=16):
    group = group.sort_values('date')  # Ensure the group is sorted by timestamp
    sequences = [group.iloc[i:i+seq_length] for i in range(0, len(group)-seq_length)]
    return dd.concat(sequences)

# Step 4: Apply the create_sequences function to each group
#sequences = grouped.apply(create_sequences, meta=ddf, seq_length=16)
sequences = ddf.loc[(ddf.store_nbr==1) & (ddf.family<=2)].groupby('store_nbr','family').apply(create_sequences, meta=ddf.loc[(ddf.store_nbr==1) & (ddf.family<=2)],seq_length=16)

# Step 5: Compute the result to get the sequences as a Dask DataFrame
sequences = sequences.compute()

In [ ]:
ddf = dd.from_pandas(data, npartitions=4)  
ddf

In [ ]:
from dask.array.lib.stride_tricks import sliding_window_view
import dask.array as da

#dask_array = da.from_array(ddf.values)
# Assuming you want sequences of length 10
seq_length = 16
sequences = sliding_window_view(ddf.values, (seq_length, data.shape[1]))


In [ ]:
X_trainM, X_testM, X_valM, y_trainM, y_testM, y_valM, base_trainM, base_testM, base_valM = [],[],[],[],[],[],[],[],[]
init=False
for family in data.family.unique():
    for store in data.store_nbr.unique():
        group = data.loc[(data.family==family) & (data.store_nbr == store)]
        X_trainM0, X_testM0, X_valM0, y_trainM0, y_testM0, y_valM0, base_trainM0, base_testM0, base_valM0 = getSequencePerGroup(group)
 
        if init:
            X_trainM = np.concatenate((X_trainM, X_trainM0), axis=0)
            X_testM  = np.concatenate((X_testM,  X_testM0), axis=0)
            X_valM  = np.concatenate((X_valM,  X_valM0), axis=0)
            y_trainM = np.concatenate((y_trainM, y_trainM0), axis=0)
            y_testM  = np.concatenate((y_testM,  y_testM0), axis=0)
            y_valM  = np.concatenate((y_valM,  y_valM0), axis=0)
            base_trainM = np.concatenate((base_trainM, base_trainM0), axis=0)
            base_testM  = np.concatenate((base_testM,  base_testM0), axis=0)
            base_valM  = np.concatenate((base_valM,  base_valM0), axis=0)
        else:
            X_trainM, X_testM, X_valM, y_trainM,y_testM, y_valM = X_trainM0, X_testM0, X_valM0,y_trainM0, y_testM0, y_valM0
            base_trainM, base_testM, base_valM = base_trainM0, base_testM0, base_valM0
            init=True 

In [ ]:
X_trainM.shape

In [ ]:
X_trainM.shape, y_trainM.shape, base_trainM.shape,X_testM.shape, y_testM.shape, base_testM.shape,X_valM.shape, y_valM.shape, base_valM.shape

In [ ]:
y_valM

# LGBM t-16 fits
- seems that families are too different, we optimize only a couple families but others are declining -> lgbm per family

In [ ]:
def calcLossArima(pred, y, logTransform):
    if logTransform:
        a = np.exp(pred) -1
        y = np.exp(y) -1 
    else:
        a = (pred)
        y = (y)

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)

    rmsleTrain = np.sqrt(mean_squared_log_error(a,y))
    return rmsleTrain

In [ ]:
def calcLossLGBM(pred, y):
    logPred = np.reshape(pred, (pred.shape[0],1))
    a = np.exp(logPred) -1

    logy = np.reshape(y, (pred.shape[0],1))
    y = np.exp(logy) -1

    rmsleTrain = np.sqrt(np.mean((a-y)**2))
    return rmsleTrain
def mse(pred,y):
    return np.sqrt(np.mean((pred-y)**2))
def calcLossLGBMArima1(pred, sales, arima, predictDiff):
    if predictDiff:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
    a = np.exp(pred) -1

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)
    rmsleTrain = np.sqrt(mean_squared_log_error(a,sales))
    return rmsleTrain
def calcLossLGBMArima2(pred, y, arima, predictDiff):
    if predictDiff:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
        y = np.reshape(y, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
    a = np.exp(pred) -1
    y = np.exp(y) -1

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)
    rmsleTrain = np.sqrt(mean_squared_log_error(a,y))
    return rmsleTrain
def plotLGBM(i, logTransform, salesDomain, pred, y, base):
    if salesDomain:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(base, (pred.shape[0],1))
        y = np.reshape(y, (pred.shape[0],1)) + np.reshape(base, (pred.shape[0],1))
    if logTransform:
        a = np.exp(pred[i:i+16])-1
        y = np.exp(y[i:i+16]) -1
        arima = np.exp(base[i:i+16]) - 1
    else:
        a = (pred[i:i+16])
        y = (y[i:i+16])
        arima = base[i:i+16]
    x = range(len(a))   

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
    axs.plot(x, y, color='blue',label='Original')
    axs.plot(x, a, color='red',label='pred')
    axs.plot(x, arima, color='orange',label='arima')
    axs.set_title('index: '+str(i))
    fig.subplots_adjust(hspace=0.5)


In [ ]:
 # compare against base lgbm that just predicts always t+16
sample_weights = ((train_bigDf.month > 7) & (train_bigDf.month < 9)) + 1# (train_bigDf.date > "2017-06-01") + 1 #train_bigDf.logSales+1 #np.reshape(( * train_bigDf.onpromotion.std()) + train_bigDf.onpromotion.mean()+ 1, (-1,1))
sample_weights = sample_weights/ max(sample_weights)



# Set parameters for LGBM model
params = {
    'boosting':'gbdt',#'gbdt', #'rf' #'dart'
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'mse',  # Mean squared error
    'num_leaves': 10,
    #'lambda_l1': 0.1,
    #'lambda_l2': 0.2,
    #'max_depth':10,
    #'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'force_col_wise':True,
}   

# Train the model
gbm = lgb.train(params, lgb.Dataset(X_train16, label=y_train16
    #, weight=sample_weights
    ), 2000,valid_sets=[lgb.Dataset(X_test16, label=y_test16)]#,num_boost_round=100
,callbacks=[lgb.early_stopping(stopping_rounds=500)]
)  
predtrainLGBM = gbm.predict(X_train16)
predtestLGBM = gbm.predict(X_test16)
predvalLGBM = gbm.predict(X_pred16)

#print('arima errors: ', calcLossArima(train_subDf.ref, train_subDf.logSales, logTransform), calcLossArima(test_subDf.ref, test_subDf.logSales, logTransform))#, calcLossArima(val_subDf.ref, val_subDf.logSales, logTransform, baseTest))
print('arima errors: ', calcLossArima(train_bigDf.ref, train_bigDf.logSales, logTransform16), calcLossArima(test_bigDf.ref, test_bigDf.logSales, logTransform16))
#print('lgbm errors: ', calcLossLGBM(predtrainLGBM, y_train), calcLossLGBM(predtestLGBM, y_test))#, calcLossLGBM(predvalLGBM, y_val))

print('sales errors: ', mse(predtrainLGBM, y_train16), mse(predtestLGBM, y_test16))
print('sales errors: ', calcLossLGBMArima2(predtrainLGBM, y_train16, baseTrain16, predictDiff16), calcLossLGBMArima2(predtestLGBM, y_test16, baseTest16,predictDiff16))#, calcLossLGBMArima2(predvalLGBM, y_val,baseVal))
#print('sales errors: ', calcLossLGBMArima1(predtrainLGBM, train_subDf.sales, baseTrain), calcLossLGBMArima1(predtestLGBM, test_subDf.sales, baseTest), calcLossLGBMArima1(predvalLGBM, val_subDf.sales,baseVal))

#sales errors:  0.3925699715971475 0.38953956730324635 with weight on date & groupedF
#sales errors:  0.389990522698507 0.387005544750438    without weight on date

# sales errors:  0.38269330293604925 0.38686356026792235 without weight & more features
# sales errors:  0.3838912161743427  0.3874510405251872  with weight on recent
# sales errors:  0.406618874649369   0.3908985813082843  with weight on juli, august, september
# sales errors:  0.38151648760876566 0.3880183808501404  with weight on august
# sales errors:  0.3809698888298268  0.3878809225435719  less lags / feature lags no weight

In [ ]:
print('arima errors: ', calcLossArima(train_bigDf.ref, train_bigDf.logSales, logTransform16), calcLossArima(test_bigDf.ref, test_bigDf.logSales, logTransform16))
#print('lgbm errors: ', calcLossLGBM(predtrainLGBM, y_train), calcLossLGBM(predtestLGBM, y_test))#, calcLossLGBM(predvalLGBM, y_val))

print('sales errors: ', mse(predtrainLGBM, y_train16), mse(predtestLGBM, y_test16))
print('sales errors: ', calcLossLGBMArima2(predtrainLGBM, y_train16, baseTrain16, predictDiff16), calcLossLGBMArima2(predtestLGBM, y_test16, baseTest16,predictDiff16))#, calcLossLGBMArima2(predvalLGBM, y_val,baseVal))


### analyze residuals

In [ ]:
def plotDistOfTestTrain(feat, test, train,labelA ='test',labelB='train'):
    a = test[[feat]].value_counts()
    b = train[[feat]].value_counts()

    indexa, indexb = [],[]
    for i, val in enumerate(a.index):
        indexa.append(val[0])
    for i, val in enumerate(b.index):
        indexb.append(val[0])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(indexa, a.values/np.sum(a), alpha=0.5, label=labelA)
    ax.bar(indexb, b.values/np.sum(b), alpha=0.5, label=labelB)
    ax.set_title('Distribution of' + feat)
    ax.set_xlabel('Category')
    ax.set_ylabel('Frequency')
    plt.xticks(rotation=90)
    ax.legend()
    plt.show()

def display2Ddifferences(feata, featb, dfa, dfb):
    a = dfa[[feata,featb]].value_counts().sort_index()
    b = dfb[[feata,featb]].value_counts().sort_index()
    xa, xb = [],[]
    ya, yb = [],[]
    for i, val in enumerate(a.index):
        xa.append(val[0])
        ya.append(val[1])
    for i, val in enumerate(b.index):
        xb.append(val[0])
        yb.append(val[1])

    df1 = pd.DataFrame()
    df1[feata] = xa
    df1[featb]=ya
    df1['f'] = a.values/np.sum(a)*100

    df2 = pd.DataFrame()
    df2[feata] = xb
    df2[featb]=yb
    df2['f'] = b.values/np.sum(b)*100


    fig = px.scatter(df1,
        y=feata,
    x=featb,
        color='f',title='bad'
    )
    fig.show()

    fig1 = px.scatter(df2,
        y=feata,
        x=featb,
        color='f',title='good'
    )
    fig1.show()

    return df1, df2


In [ ]:
train_bigDf['predTarget'] = predtrainLGBM
test_bigDf['predTarget'] = predtestLGBM
train_bigDf['err_predTarget'] = np.abs(train_bigDf['predTarget'] - train_bigDf['target'])
test_bigDf['err_predTarget'] = np.abs(test_bigDf['predTarget'] - test_bigDf['target'])



train_bigDf['predLogSales'] = train_bigDf['predTarget']+train_bigDf['salesArima']
test_bigDf['predLogSales'] = test_bigDf['predTarget']+test_bigDf['salesArima']
train_bigDf['err_LogSales'] = (train_bigDf['predLogSales'] - train_bigDf['logSales'])**2
test_bigDf['err_LogSales'] =(test_bigDf['predLogSales'] - test_bigDf['logSales'])**2
train_bigDf['err_arimaLogSales'] = (train_bigDf['salesArima'] - train_bigDf['logSales'])**2
test_bigDf['err_arimaLogSales'] = (test_bigDf['salesArima'] - test_bigDf['logSales'])**2

# arimaLogError > errLogSales = positive (= reduction of errLogSales)
train_bigDf['improvedScoreByLGBM'] = train_bigDf['err_arimaLogSales'] - train_bigDf['err_LogSales']
test_bigDf['improvedScoreByLGBM'] = test_bigDf['err_arimaLogSales'] - test_bigDf['err_LogSales']

train_bigDf['predSales'] = np.exp(train_bigDf.predLogSales) -1
test_bigDf['predSales'] = np.exp(test_bigDf.predLogSales) -1#
train_bigDf['err_sales'] = np.abs(train_bigDf['predSales'] - train_bigDf['sales'])
test_bigDf['err_sales'] = np.abs(test_bigDf['predSales'] - test_bigDf['sales'])

train_bigDf['predArimaSales'] = np.exp(train_bigDf.salesArima) -1
test_bigDf['predArimaSales'] = np.exp(test_bigDf.salesArima) -1
train_bigDf['err_salesArima'] = np.abs(train_bigDf['predArimaSales'] - train_bigDf['sales'])
test_bigDf['err_salesArima'] = np.abs(test_bigDf['predArimaSales'] - test_bigDf['sales'])

In [ ]:
train_bigDf[['err_LogSales','err_arimaLogSales','improvedScoreByLGBM']].hist(bins=50),test_bigDf[['err_LogSales','err_arimaLogSales','improvedScoreByLGBM']].hist(bins=100)

#### what got worse

In [ ]:
# err_logSales = (logPred - arimaLog - logSales + arimaLog)²
# err_arimaLogSales = (arimaLog - logSales)²
test_bigDf.sort_values(by='improvedScoreByLGBM')  # which got worse

In [ ]:
badPredictionsTest = test_bigDf.loc[test_bigDf.improvedScoreByLGBM>0]
goodPredictionsTest = test_bigDf.loc[test_bigDf.improvedScoreByLGBM<0]
badPredictionsTrain = train_bigDf.loc[train_bigDf.improvedScoreByLGBM>0]
goodPredictionsTrain = train_bigDf.loc[train_bigDf.improvedScoreByLGBM<0]

#mainly family 29 & 17 got better

In [ ]:
# only family & store seem to have significant differences between good/bad
plotDistOfTestTrain('family',badPredictionsTest,goodPredictionsTest,'bad','good')
plotDistOfTestTrain('store_nbr',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('onpromotion',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('city',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('state',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('type',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('cluster',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('weekday',badPredictionsTest,goodPredictionsTest,'bad','good')

In [ ]:
plotDistOfTestTrain('family',badPredictionsTrain,goodPredictionsTrain,'bad','good')
plotDistOfTestTrain('store_nbr',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('onpromotion',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('city',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('state',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('type',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('cluster',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('weekday',badPredictionsTrain,goodPredictionsTrain,'bad','good')

#### what is generally bad
families
- 29,17,14,13 are good - 29&17 are often 0
- 31,18,12,0 bad

In [ ]:
test_bigDf.sort_values(by='err_LogSales')

In [ ]:
badPredictionsTest = test_bigDf.loc[test_bigDf.err_LogSales>1]
goodPredictionsTest = test_bigDf.loc[test_bigDf.err_LogSales<1]
badPredictionsTrain = train_bigDf.loc[train_bigDf.err_LogSales>1]
goodPredictionsTrain = train_bigDf.loc[train_bigDf.err_LogSales<1]

# bad families: 0,2,6,7,10-14,16,18,19,20,27,31,32
# bad stores: 0,10-23,25,26,28-36,39,40,43,51

In [ ]:
flippedPropDicts['family']

In [ ]:
badFreq, goodFreq = display2Ddifferences('store_nbr','family', badPredictionsTest, goodPredictionsTest)

In [ ]:
freqDf = pd.merge(badFreq, goodFreq, on=['store_nbr','family'], how='outer',suffixes=('bad','good'))
freqDf['diff'] = freqDf.fgood-freqDf.fbad
freqDf.sort_values(by='diff').head(20)

In [ ]:
train_bigDf.loc[(train_bigDf.family==0) & (train_bigDf.store_nbr==48)].sales.plot()

In [ ]:
# only family & store seem to have significant differences between good/bad
plotDistOfTestTrain('family',badPredictionsTest,goodPredictionsTest,'bad','good')
plotDistOfTestTrain('store_nbr',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('onpromotion',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('city',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('state',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('type',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('cluster',badPredictionsTest,goodPredictionsTest,'bad','good')
#plotDistOfTestTrain('weekday',badPredictionsTest,goodPredictionsTest,'bad','good')

In [ ]:
plotDistOfTestTrain('family',badPredictionsTrain,goodPredictionsTrain,'bad','good')
plotDistOfTestTrain('store_nbr',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('onpromotion',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('city',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('state',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('type',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('cluster',badPredictionsTrain,goodPredictionsTrain,'bad','good')
#plotDistOfTestTrain('weekday',badPredictionsTrain,goodPredictionsTrain,'bad','good')

#### is test/train different - Answer:no

In [ ]:
plotDistOfTestTrain('family',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('store_nbr',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('onpromotion',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('city',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('state',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('type',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('cluster',badPredictionsTest,badPredictionsTrain)
plotDistOfTestTrain('weekday',badPredictionsTest,badPredictionsTrain)
